### Consigna: Utilice los textos del contenido para realizar consultas por texto utilizando las técnicas vistas en la clase de NLP (BOW o TF-IDF) de modo que la query "retablo iglesia" devuelva alguna página acerca del retablo de alguna iglesia (⭐⭐)

Importo las librerias que utilizaré para resolver el ejercicio.

In [8]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords
import numpy as np

In [9]:
#Leere solo las columnas que me interesan para no ocupar espacio demas en la RAM
pages = pd.read_csv('contents_text_sample.csv')
pages

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...
...,...,...,...
206602,10010394,Nicoleta-Ancuţa Bodnar,#REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
206603,10010425,Sebastian Szczepański,{{Ficha de deportista\n|nombre = ...
206604,10010518,Manu (Tripura),{{Ficha de entidad subnacional\n|nombre comple...
206605,10010570,ISRU,#REDIRECCIÓN [[Utilización de recursos in situ]]


In [10]:
pages 

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...
...,...,...,...
206602,10010394,Nicoleta-Ancuţa Bodnar,#REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
206603,10010425,Sebastian Szczepański,{{Ficha de deportista\n|nombre = ...
206604,10010518,Manu (Tripura),{{Ficha de entidad subnacional\n|nombre comple...
206605,10010570,ISRU,#REDIRECCIÓN [[Utilización de recursos in situ]]


In [11]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206607 entries, 0 to 206606
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      206607 non-null  int64 
 1   title   206607 non-null  object
 2   text    206600 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.7+ MB


In [12]:
pages.isnull().sum()

id       0
title    0
text     7
dtype: int64

In [13]:
pages.fillna('Desconocido',inplace=True)

In [14]:
pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206607 entries, 0 to 206606
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      206607 non-null  int64 
 1   title   206607 non-null  object
 2   text    206607 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.7+ MB


El dataFrame ya esta en optimas condiciones para ser utilizado

In [15]:
#guardo en una variable las stop_words españolas para trabajar con ellas
stop_words = nltk.corpus.stopwords.words('spanish')
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [16]:
#creo el objeto Vectorizer para buscar las palabras requeridas en los textos
vectorizer = TfidfVectorizer(lowercase=True,stop_words=stop_words,
                            max_features=10000)

In [17]:
matrix = vectorizer.fit_transform(pages['text'])

In [18]:
matrix

<206607x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 18077731 stored elements in Compressed Sparse Row format>

In [19]:
#Utilizo el NearestNeighbors para buscar una pagina cercana al retablo de una iglesia
buscador = NearestNeighbors(algorithm='brute', metric='cosine')
buscador.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
query = 'retablo iglesia'
vector = vectorizer.transform([query])
dist, indexes = buscador.kneighbors(vector, n_neighbors=10)
for d, i in zip(dist.flatten(), indexes.flatten()):
    print("A distancia %f el texto: %s" % (d,pages['text'].iloc[i]))

A distancia 0.164440 el texto: #REDIRECCIÓN [[Retablo mayor de la iglesia de San Millán de los Balbases]]
A distancia 0.187607 el texto: #REDIRECCIÓN [[Retablo de Bernabé]]
A distancia 0.187607 el texto: #REDIRECCIÓN [[Retablo]]
A distancia 0.225645 el texto: #REDIRECT [[Retablo]]
A distancia 0.273681 el texto: #REDIRECCIÓN  [[Retablo mayor de la Basilica de Lequeitio]]
A distancia 0.357811 el texto: #REDIRECCIÓN [[Retablo de la Virgen de los Ediles]]
A distancia 0.447834 el texto: {{Ficha de película
| título original = Retablo
| dirección = Álvaro Delgado-Aparicio
| producción = SIRI Producciones
| guión = Álvaro Delgado Aparicio, Héctor Gálvez
| música = Harry Escott
| fotografía = Mario Bassino
| reparto = [[Júnior Béjar]]<br/>Amiel Cayo<br/>[[Magaly Solier]]
| país = {{PER}}
| año = 2017
| estreno = 2018
}}'''''Retablo''''' es una [[película dramática]] [[Cine de Perú|peruana]], debut del director [[Álvaro Delgado-Aparicio]] y estrenada en [[2017]] en el [[Festival de Cine de Lima

In [21]:
indexes

array([[193679, 132997,   5074, 114864,  50461, 140413, 176055,  82465,
          6006, 130864]], dtype=int64)

### Resultado

El resultado obtenido finalmente es el siguiente:

In [22]:
pages.iloc[193672]

id                                  9357707
title                      Torre de Cambrón
text     #REDIRECCIÓN [[Torre del Cambrón]]
Name: 193672, dtype: object